In [304]:
# import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Model from Scikit-Learn
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import tree

#Model Evaluations
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import plot_roc_curve

In [305]:

df=pd.read_csv('healthcare-dataset-stroke-data.csv')

In [306]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [307]:
df.shape

(5110, 12)

In [308]:
df.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [309]:
NumericalColumns = ['bmi']
#NumericalColumns = ['age','hypertension','heart_disease','avg_glucose_level','bmi','stroke']
#CategoricalColumns = ['gender','ever_married','work_type','Residence_type','smoking_status']

In [310]:
for columnName in NumericalColumns:
  df[columnName].fillna(df[columnName].mean(), inplace=True)

In [311]:
df.isna().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [312]:
encodeColumn = ['gender','ever_married','work_type','Residence_type','smoking_status']
df = pd.get_dummies(df , columns=encodeColumn , prefix=encodeColumn , drop_first=True)

In [313]:
df

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,67.0,0,1,228.69,36.600000,1,1,0,1,0,1,0,0,1,1,0,0
1,51676,61.0,0,0,202.21,28.893237,1,0,0,1,0,0,1,0,0,0,1,0
2,31112,80.0,0,1,105.92,32.500000,1,1,0,1,0,1,0,0,0,0,1,0
3,60182,49.0,0,0,171.23,34.400000,1,0,0,1,0,1,0,0,1,0,0,1
4,1665,79.0,1,0,174.12,24.000000,1,0,0,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,80.0,1,0,83.75,28.893237,0,0,0,1,0,1,0,0,1,0,1,0
5106,44873,81.0,0,0,125.20,40.000000,0,0,0,1,0,0,1,0,1,0,1,0
5107,19723,35.0,0,0,82.99,30.600000,0,0,0,1,0,0,1,0,0,0,1,0
5108,37544,51.0,0,0,166.29,25.600000,0,1,0,1,0,1,0,0,0,1,0,0


In [314]:
df = df.sample(frac = 1).reset_index()

X=df.drop('stroke',axis=1)
Y=df['stroke']
min_max = MinMaxScaler()
min_max.fit(X)
X = min_max.transform(X)

In [334]:

target=df['stroke']
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.30,random_state=7, stratify=target)

In [345]:
#Put Models in a dictonary

models={
    "Random Forest": RandomForestClassifier(),
    #"Decission Tree":tree.DecisionTreeClassifier(),
    "Support Vector Machines": svm.LinearSVC(),
}


def fit_and_score(models,x_train,x_test,y_train,y_test):
    model_scores={}
    #np.random.seed(42)
    Predict={}
    for name,model in models.items():
        model.fit(x_train,y_train)
        model_scores[name]=model.score(x_test,y_test)
        Predict[name]=model.predict(x_test)
        model_scores[name+" Train "]=model.score(x_train,y_train)
        
    return (model_scores,Predict)

In [381]:
model_scores,Predict=fit_and_score(
    models=models,
    x_train=X_train,
    x_test=X_test,
    y_train=Y_train,
    y_test=Y_test)
#model_scores

In [382]:
# model_compare=pd.DataFrame(model_scores,index=['accuracy'])
# model_compare.T.plot.bar()

In [383]:
#for Random Forest

comprison={}

cm = confusion_matrix(Y_test, Predict['Random Forest'])
TN, FP, FN, TP = confusion_matrix(Y_test, Predict['Random Forest']).ravel()

In [384]:
print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  75
False Positive(FP) =  0
True Negative(TN)  =  1458
False Negative(FN) =  0


In [385]:
accuracy={}
F1={}
df_model = pd.DataFrame(index=models.keys())
accuracy['Random Forest']=(TP+TN)/(TP+FP+TN+FN)
precision = (TP)/( TP + FP )
recall = (TP)/( TP + FN )
specificity = TN / ( FP + TN )
F1['Random Forest'] = (2*precision*recall)/(precision+recall)
# df_model['Accuracy']=accuracy
# df_model['F1 Score']=F1


In [386]:
#for Random Forest

cm = confusion_matrix(Y_test, Predict['Support Vector Machines'])
TN, FP, FN, TP = confusion_matrix(Y_test, Predict['Support Vector Machines']).ravel()

In [387]:
print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

True Positive(TP)  =  71
False Positive(FP) =  7
True Negative(TN)  =  1451
False Negative(FN) =  4


In [388]:
accuracy['Support Vector Machines']=(TP+TN)/(TP+FP+TN+FN)
precision = (TP)/( TP + FP )
recall = (TP)/( TP + FN )
specificity = TN / ( FP + TN )
F1['Support Vector Machines'] = (2*precision*recall)/(precision+recall)

df_model['Accuracy']=accuracy.values()
df_model['F1 Score']=F1.values()


In [389]:
#draw Co

In [390]:

df_model

,Accuracy,F1 Score
Random Forest,1.000000,1.000000
Support Vector Machines,0.992825,0.928105
